In [ ]:
from collections import Counter

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
SAMPLE_INDEX = 9

In [ ]:
def get_headerless_vcf_df(f_obj, stop_on='#CHROM'):
    line = ''
    header = ''
    while True:
        line = f_obj.readline()
        header += line
        if line[:6] == stop_on:
            break
            
#     print(line)
            
    df = pd.read_csv(f_obj, sep='\t', header=None)
    df.columns = line[1:].replace('""', '').replace('\n', '').split('\t')

    return df, header

In [ ]:
def get_calls_matrix(vcf_fp):
    f_obj = open(vcf_fp)
    
    line = ''
    while True:
        line = f_obj.readline()
        if line[:6] == '#CHROM':
            break
    
    line = line.replace('\n', '')
    samples = line.split('\t')[SAMPLE_INDEX:]
    
    X = []
    for line in f_obj:
        line = line.replace('\n', '')
        pieces = line.split('\t')
        X.append(pieces[SAMPLE_INDEX:])
    
    return np.asarray(X), samples

### assumptions

sample vcf is same length as genomes vcf

In [ ]:
# CALLED_VCF_FP = '/diskmnt/Projects/Users/estorrs/data/ancestry/MM/temp/called_samples.vcf'
# GENOMES_VCF_FP = '/diskmnt/Projects/Users/estorrs/1000-genomes/GRCh37/all.coding.sorted.02maf.10000sampled.sorted.snps.vcf'

In [ ]:
CALLED_VCF_FP = '/gscmnt/gc2737/ding/estorrs/ancestry/MMRF/vcf/called.vcf'
MM_CALLED_VCF_FP = '/gscmnt/gc2737/ding/estorrs/ancestry/MM/vcf/called.vcf'
GENOMES_VCF_FP = '/gscmnt/gc2737/ding/estorrs/1000-genomes/GRCh37/all.coding.sorted.02maf.10000sampled.sorted.snps.vcf'
PREDICTIONS_FP = '/gscmnt/gc2737/ding/estorrs/ancestry/MMRF/predictions/predictions.tsv'
SELF_REPORTED_FP = '/gscmnt/gc2737/ding/estorrs/ancestry/MMRF/predictions/mmrf_self_reported.tsv'
MM_PAPER_SAMPLES_FP = '/gscmnt/gc2737/ding/estorrs/ancestry/MMRF/predictions/MM_paper_patients.tsv'

In [ ]:
# def preprocess_vcf(vcf_fp):
#     """Pull out X, convert to 
#     Returns:
#         (X, scaler)
#     """
#     calls_matrix, samples = get_calls_matrix(vcf_fp)
    
#     for i, row in enumerate(calls_matrix):
#         row = [re.sub(re.compile(r'^0\|[1-9]+|^[1-9]+\|0'), r'0|1', v) for v in row]
#         calls_matrix[i] = [re.sub(re.compile(r'^[1-9]+\|[1-9]+'), r'1|1', v) for v in row]
        
#     # encode calls
#     label_encoder = LabelEncoder()
#     label_encoder.fit(['.|.', '0|0', '0|1', '1|1', '0', '1'])
    
#     for i, row in enumerate(calls_matrix):
#         calls_matrix[i] = label_encoder.transform(row)
    
# #     print(ca)
    
#     calls_matrix = np.transpose(calls_matrix)
    
#     return calls_matrix, samples

def get_preprocessed_X(df):    
#     if drop_indices is None:
        
    
    # remove x
    df = df[df['CHROM'] != 'X']
    df = df[df['CHROM'] != 'chrX']
    
    trimmed_df = df[df.columns[9:]]
    samples = trimmed_df.columns
    
    trimmed_df = trimmed_df.replace(re.compile(r'^0\|[1-9]+|^[1-9]+\|0'), '0|1')
    trimmed_df = trimmed_df.replace(re.compile(r'^[1-9]+\|[1-9]+'), '1|1')
    
#     return trimmed_df
    # for X chrom
#     trimmed_df = trimmed_df.replace(re.compile(r'^[1-9]+$'), '1')
    
    # encode genotype
    label_encoder = LabelEncoder()
#     label_encoder.fit(['.|.', '0|0', '0|1', '1|1', '0', '1'])
    label_encoder.fit(['.|.', '0|0', '0|1', '1|1'])

    # encode the rows
    for i, row in trimmed_df.iterrows():
        trimmed_df.loc[i][:] = label_encoder.transform(row.values)
        
    X = trimmed_df.values

    # transpose so each row is now a sample
    X = X.transpose()
    
    return X, label_encoder, samples, trimmed_df
    

In [ ]:
df_genomes, _ = get_headerless_vcf_df(open(GENOMES_VCF_FP))
df_called, _ = get_headerless_vcf_df(open(CALLED_VCF_FP))
df_called_MM, _ = get_headerless_vcf_df(open(MM_CALLED_VCF_FP))

In [ ]:
# drop rows with all missing
to_drop = []
for i, row in df_called.iterrows():
    if len([x for x in list(row[9:]) if x != '.|.']) <= len(row[9:]) * .95:
        to_drop.append(i)
        
for i, row in df_called_MM.iterrows():
    if len([x for x in list(row[9:]) if x != '.|.']) <= len(row[9:]) * .95:
        to_drop.append(i)

df_genomes = df_genomes.drop(to_drop) 
df_called = df_called.drop(to_drop)
df_called_MM = df_called_MM.drop(to_drop)

In [ ]:
df_genomes.shape, df_called.shape, df_called_MM.shape

In [ ]:
X_genomes_, encoder_genomes, samples_genomes, trimmed_df_genomes = get_preprocessed_X(df_genomes)

In [ ]:
X_called_, encoder_called, samples_called, trimmed_df_called = get_preprocessed_X(df_called)

In [ ]:
X_called_MM_, encoder_called_MM, samples_called_MM, trimmed_df_called_MM = get_preprocessed_X(df_called_MM)

In [ ]:
# make copys so you dont have to rerun a bunch of times
X_genomes = np.copy(X_genomes_)
X_called = np.copy(X_called_)
X_called_MM = np.copy(X_called_MM_)

In [ ]:
# impute missing values
imp_most_freq = SimpleImputer(missing_values=0, strategy='most_frequent')
X_genomes = imp_most_freq.fit_transform(X_genomes)
X_called = imp_most_freq.fit_transform(X_called)
X_called_MM = imp_most_freq.fit_transform(X_called_MM)

In [ ]:
X_genomes.shape, X_called.shape, X_called_MM.shape

In [ ]:
np.all(df_genomes['POS'] == df_called['POS'])

In [ ]:
pca_scaler = StandardScaler()
pca_scaler.fit(X_genomes)
X_genomes = pca_scaler.transform(X_genomes)

In [ ]:
X_called = pca_scaler.transform(X_called)

In [ ]:
X_called_MM = pca_scaler.transform(X_called_MM)

In [ ]:
# print(X_genomes.shape, X_called.shape)
# X = np.vstack((X_genomes, X_called))
# print(X.shape)

## PCA

In [ ]:
# pca_scaler = StandardScaler()
# pca_scaler.fit(X)
# X = pca_scaler.transform(X)

In [ ]:
# pca = PCA(n_components=20)
# pca.fit(X)
# pcs = pca.transform(X)

In [ ]:
# pcs_df = pd.DataFrame(pcs)
# pd.scatter_matrix(pcs_df.loc[:, :3], figsize=(12,12))

In [ ]:
pca = PCA(n_components=20)
pca.fit(X_genomes)
pcs_genomes = pca.transform(X_genomes)

In [ ]:
pcs_genomes_df = pd.DataFrame(pcs_genomes)
pd.scatter_matrix(pcs_genomes_df.loc[:, :3], figsize=(12,12))

In [ ]:
pcs_called = pca.transform(X_called)

In [ ]:
pcs_called_df = pd.DataFrame(pcs_called)
pd.scatter_matrix(pcs_called_df.loc[:, :3], figsize=(12,12))

In [ ]:
pcs_called_MM = pca.transform(X_called_MM)

In [ ]:
pcs_called_MM_df = pd.DataFrame(pcs_called_MM)
pd.scatter_matrix(pcs_called_MM_df.loc[:, :3], figsize=(12,12))

In [ ]:
# component_df = pd.DataFrame(pca.components_,columns=trimmed_df_genomes.index)

In [ ]:
# def get_most_informative_variants(df, variant_)

In [ ]:
# idxs = np.argsort(np.asarray([abs(x) for x in pca.components_[0, :]]))
# [idxs[-10:]]

## annotate 1000 genomes

In [ ]:
# load in 1000 genome labels
labels_df = pd.read_csv('/gscmnt/gc2737/ding/estorrs/1000-genomes/GRCh37/integrated_call_samples_v3.20130502.ALL.panel',
                       sep='\t')
labels_df.head()

In [ ]:
# make sure order still matches
np.all(df_genomes.columns[9:] == labels_df['sample'])

In [ ]:
sample_to_ancestry = {k:v for k, v in zip(labels_df['sample'], labels_df['super_pop'])}

ancestry_to_color = {'EUR': 'red', 'EAS': 'blue', 'AMR': 'green', 'AFR': 'orange', 'SAS': 'purple'}
colors = [ancestry_to_color[sample_to_ancestry[s]] for s in labels_df['sample']]

In [ ]:
# ## extract back out into called and 1000 genomes
# pcs_genomes_df = pcs_df[:X_genomes.shape[0]][:]
# pcs_called_df = pcs_df[X_genomes.shape[0]:][:]
# pcs_genomes_df.shape, pcs_called_df.shape

In [ ]:
def get_axis_limits(pcs_df, d=4):
    lims = []
    for i in range(d):
        c = pcs_df[i]
        lims.append((min(c), max(c)))
        
    return lims
        

In [ ]:
pd.scatter_matrix(pcs_genomes_df.loc[:, :3], color=colors, figsize=(12,12))

In [ ]:
lims = get_axis_limits(pcs_genomes_df)
lims

In [ ]:
ax = pd.plotting.scatter_matrix(pcs_called_df.loc[:, :3], figsize=(12,12), diagonal='kde')
# for i, (ax_min, ax_max) in enumerate(lims):
#     for j in range(len(lims)):
#         ax[i, j].set_ylim(ax_min, ax_max)
#         ax[j, i].set_xlim(ax_min, ax_max)

## do classifier

In [ ]:
# get target values
ancestries = labels_df['super_pop'].values
ancestry_encoder = LabelEncoder()
ancestry_encoder.fit(list(set(ancestries)))
y = ancestry_encoder.transform(ancestries)

In [ ]:
# do normalization
X = np.copy(pcs_genomes_df.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
rf_scaler = StandardScaler()
rf_scaler.fit(X_train)
X_train = rf_scaler.transform(X_train)
X_test = rf_scaler.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

In [ ]:
# train accuracy and test accuracy
clf.score(X_train, y_train), clf.score(X_test, y_test)

## classify samples

In [ ]:
# do classification of mm samples
samples = df_called.columns[9:]
X_samples = np.copy(pcs_called_df.values)
X_samples = rf_scaler.transform(X_samples)

predictions = clf.predict(X_samples)
predictions = ancestry_encoder.inverse_transform(predictions)

sample_to_predictions = {k:v for k, v in zip(samples, predictions)}
colors = [ancestry_to_color[sample_to_predictions[s]] for s in samples]

In [ ]:
ax = pd.plotting.scatter_matrix(pcs_called_df.loc[:, :3], color=colors, figsize=(12,12), diagonal='kde')
# for i, (ax_min, ax_max) in enumerate(lims):
#     for j in range(len(lims)):
#         ax[i, j].set_ylim(ax_min, ax_max)
#         ax[j, i].set_xlim(ax_min, ax_max)

In [ ]:
# do classification of mm 14 samples
samples_MM = df_called_MM.columns[9:]
X_samples_MM = np.copy(pcs_called_MM_df.values)
X_samples_MM = rf_scaler.transform(X_samples_MM)

predictions_MM = clf.predict(X_samples_MM)
predictions_MM = ancestry_encoder.inverse_transform(predictions_MM)

sample_to_predictions_MM = {k:v for k, v in zip(samples_MM, predictions_MM)}
colors = [ancestry_to_color[sample_to_predictions_MM[s]] for s in samples_MM]

In [ ]:
ax = pd.plotting.scatter_matrix(pcs_called_MM_df.loc[:, :3], color=colors, figsize=(12,12), diagonal='kde')
for i, (ax_min, ax_max) in enumerate(lims):
    for j in range(len(lims)):
        ax[i, j].set_ylim(ax_min, ax_max)
        ax[j, i].set_xlim(ax_min, ax_max)

In [ ]:
sample_prediction_tups = [(s, p) for s, p in sample_to_predictions.items()]
sample_prediction_tups = sorted(sample_prediction_tups, key=lambda x: x[0])
samps, preds = zip(*sample_prediction_tups)
prediction_df = pd.DataFrame.from_dict({
    'sample': samps,
    'predicted_ancestry': preds
})
prediction_df.head()

In [ ]:
prediction_df.to_csv(PREDICTIONS_FP, sep='\t', index=False)

## nice plot

In [ ]:
## deduplicate samples
def deduplicate_col(df, col_name):
    to_drop = []
    prev = None
    for i, row in df.iterrows():
        if prev is not None and prev == row[col_name]:
            to_drop.append(i)

        prev = row[col_name]
    
    return df.drop(to_drop) 

def pad_ax_limits(limits, padding=.05):
    return [(a - abs(padding * (b - a)), b + abs(padding * (b - a))) for a, b in limits]

In [ ]:
def scatterplot(df, x_dim, y_dim, category, colormap, shapemap=None, figsize=(10, 5), size=5, alpha=.7, fontsize=10,
               legend_props={}, save=None, title='', xlim=None, ylim=None):
    
 
    fig, ax = plt.subplots(figsize=figsize)
    
    for cat in sorted(list(set(df[category]))):
        mini_df = df[df[category]==cat]
        x = mini_df[x_dim]
        y = mini_df[y_dim]
        
        plt.scatter(x, y, c=colormap[cat], label=cat, s=size, alpha=alpha)            

    ax.set_title(title).set_fontsize(fontsize)
    ax.set_xlabel(x_dim).set_fontsize(fontsize)
    ax.set_ylabel(y_dim).set_fontsize(fontsize)
    
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)

    ax.legend(prop=legend_props)
    
    if save is None:
        plt.show()
    else:
        plt.savefig(save, dpi=1200)

In [ ]:
padded_lims = pad_ax_limits(lims, padding=.1)
padded_lims

In [ ]:
genome_plot_df = pcs_genomes_df[[0, 1]]
genome_plot_df.columns = ['PC1', 'PC2']
genome_plot_df['Ancestry'] = list(labels_df['super_pop'])
scatterplot(genome_plot_df, 'PC1', 'PC2', 'Ancestry', ancestry_to_color, size=8, figsize=(10, 10), fontsize=20,
           legend_props={'size': 15}, title='1000 Genomes', xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
self_reported_df = pd.read_csv(SELF_REPORTED_FP, sep='\t')
self_reported_df.head()

In [ ]:
self_reported_df.columns = ['sample', 'self_reported_ancestry']
self_reported_df = self_reported_df.replace(1, 'EUR')
self_reported_df = self_reported_df.replace(2, 'AFR')
self_reported_df = self_reported_df.replace(3, 'OTHER')
self_reported_df.head()

In [ ]:
called_plot_df = pcs_called_df[[0, 1]]
called_plot_df.columns = ['PC1', 'PC2']
called_plot_df['Ancestry'] = list(predictions)

samples_called_shortened = [s[:-2] for s in samples_called]
called_plot_df['sample'] = samples_called_shortened
called_plot_df['sample_id'] = samples_called

called_plot_df = called_plot_df.merge(self_reported_df, on='sample')

deduplicated_called_plot_df = deduplicate_col(called_plot_df, 'sample')

In [ ]:
called_plot_df.shape, deduplicated_called_plot_df.shape

In [ ]:

scatterplot(deduplicated_called_plot_df, 'PC1', 'PC2', 'Ancestry', ancestry_to_color, size=8, figsize=(10, 10), fontsize=20,
           legend_props={'size': 15}, title='MMRF - predicted', alpha=.7, xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
Counter(deduplicated_called_plot_df['Ancestry'])

In [ ]:
self_reported_ancestry_to_color = {
    'EUR': ancestry_to_color['EUR'],
    'AFR': ancestry_to_color['AFR'],
    'OTHER': 'gray'
}


scatterplot(deduplicated_called_plot_df, 'PC1', 'PC2', 'self_reported_ancestry', self_reported_ancestry_to_color, size=8, figsize=(10, 10),
            fontsize=20, legend_props={'size': 15}, title='MMRF - self reported', xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
Counter(deduplicated_called_plot_df['self_reported_ancestry'])

In [ ]:
MM_paper_samples = pd.read_csv(MM_PAPER_SAMPLES_FP, sep='\t', header=None)
MM_paper_samples = list(MM_paper_samples[0])
# MM_paper_samples

In [ ]:
MMRF_paper_df = deduplicated_called_plot_df[called_plot_df['sample'].isin(MM_paper_samples)]
MMRF_paper_df.shape

In [ ]:
scatterplot(MMRF_paper_df, 'PC1', 'PC2', 'Ancestry', ancestry_to_color, size=8, figsize=(10, 10),
            fontsize=20, legend_props={'size': 15}, title='MMRF paper - predicted', xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
Counter(MMRF_paper_df['Ancestry'])

In [ ]:
scatterplot(MMRF_paper_df, 'PC1', 'PC2', 'self_reported_ancestry', ancestry_to_color, size=8, figsize=(10, 10),
            fontsize=20, legend_props={'size': 15}, title='MMRF paper - self reported', xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
called_plot_MM_df = pcs_called_MM_df[[0, 1]]
called_plot_MM_df.columns = ['PC1', 'PC2']
called_plot_MM_df['Ancestry'] = list(predictions_MM)

In [ ]:

# scatterplot(called_plot_MM_df, 'PC1', 'PC2', 'Ancestry', ancestry_to_color, size=8, figsize=(10, 10), fontsize=20,
#            legend_props={'size': 15}, title='MM - predicted', alpha=.7, xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
samples_called_MM_shortened = [s.split('-')[0] for s in samples_called_MM]
called_plot_MM_df['sample'] = samples_called_MM_shortened
called_plot_MM_df['sample_id'] = samples_called_MM

In [ ]:
scatterplot(deduplicate_col(called_plot_MM_df, 'sample'), 'PC1', 'PC2', 'Ancestry', ancestry_to_color, size=8, figsize=(10, 10), fontsize=20,
           legend_props={'size': 15}, title='MM 14 samples - predicted', alpha=.7, xlim=padded_lims[0], ylim=padded_lims[1])

In [ ]:
samples_called_MM